# Data reading and cleaning

In [2]:
# import libraries
import pandas as pd
import numpy as np
import re
pd.options.display.max_rows = 1000
pd.options.display.max_columns = 1000

In [3]:
# read "animals" csv
animals = pd.read_excel(r"C:\Users\Vanessa\Downloads\Dados\Programas\Repositório Git\NGO_DataAnalysis\raw tables\animais_2016_2019.xlsx", index_col = None)

In [4]:
# rename "animals" columns
animals.columns = animals.columns.str.lower().str.strip()

In [5]:
# read excel files after manual cleaning and join tables in a single dataset
tables_df = pd.DataFrame()
for i in range(0,82):
    a = pd.read_excel(fr"C:\Users\Vanessa\Downloads\Dados\Programas\Repositório Git\NGO_DataAnalysis\clean tables\tabela_{i}.xls")
    tables_df = pd.concat([tables_df, a], ignore_index = True)
tables_df.dtypes

ano            float64
mês             object
dia            float64
receita         object
depositante     object
despesas        object
favorecido      object
saldo           object
dtype: object

In [6]:
# create "expenses" dataframe and drop nulls
expenses = tables_df[['ano', 'mês', 'dia', 'despesas', 'favorecido']].copy()
expenses.dropna(subset = 'despesas', axis = 0, inplace=True)
expenses.reset_index(drop = True, inplace = True)

In [7]:
# create "income" dataframe and drop nulls
income = tables_df[['ano', 'mês', 'dia', 'receita', 'depositante']].copy()
income.dropna(subset = 'receita', axis = 0, inplace=True)
income.reset_index(drop = True, inplace = True)

In [8]:
# create categories for 'expenses'

mask = expenses['favorecido'].str.contains('sal|intercamb|alim|lanch|temperinho|rest|pão|café|panif|doce|confeitaria|subway|gratificação|sabor|hong ju|mansa', flags = re.I)
sal = np.where(mask, 'salários e auxílios alimentação', expenses['favorecido'])
expenses.loc[:,'favorecido'] = sal

mask2 = expenses['favorecido'].str.contains('sup|super|hiper|hipper|mercado|bistek|milium|embala|atacad|cassol|agua|distrib|loja|zeus|comerc', flags = re.I)
sup = np.where(mask2, 'suprimentos', expenses['favorecido'])
expenses.loc[:,'favorecido'] = sup

mask3 = expenses['favorecido'].str.contains('medic|clínica|clinica|vet|remédio|farm|biofilia|gral|asamed|pet|cuidados|panvel|acupuntura|raio x|praiana|castração|tosa|cremat', flags = re.I)
medic = np.where(mask3, 'medicações e veterinário', expenses['favorecido'])
expenses.loc[:,'favorecido'] = medic

mask4 = expenses['favorecido'].str.contains('hosp|poeta|lilian ribeiro|lar|cuidadora|casa|canil|canis|inse', flags = re.I)
hosp = np.where(mask4, 'hospedagem e manutenção de canis', expenses['favorecido'])
expenses.loc[:,'favorecido'] = hosp

mask5 = expenses['favorecido'].str.contains('agro|ração|reção|pecuária|terra|arcadia', flags = re.I)
rac = np.where(mask5, 'ração e agropecuária', expenses['favorecido'])
expenses.loc[:,'favorecido'] = rac

mask7 = expenses['favorecido'].str.contains('combust|escap|posto|seguro|veíc|motorista|car|borrack|park|auto|taxi|táxi|tàxi|frete|motor|estac|floripeças|multa|mecanica|mecânica|mecãnica|parachoque|borrach|estrela|localiza', flags = re.I)
trans = np.where(mask7, 'transporte', expenses['favorecido'])
expenses.loc[:,'favorecido'] = trans

mask6 = expenses['favorecido'].str.contains('camis|malhas|mat|jebelus|agenda|vest|patchwork|plumas|bazar|graf|gráf|digital|mtools|eletr|hd|site|notebook|face|cópias|copia|plot|mídia|chip|marca|mix|print', flags = re.I)
vend = np.where(mask6, 'bazar, divulgação e tecnologias', expenses['favorecido'])
expenses.loc[:,'favorecido'] = vend

mask8 = expenses['favorecido'].str.contains('empr|apli|renda', flags = re.I)
emp = np.where(mask8, 'empréstimos e aplicações', expenses['favorecido'])
expenses.loc[:,'favorecido'] = emp

mask9 = expenses['favorecido'].str.contains('tarif|INSS|IR|taxa|IPVA|DARF|celesc|tim|distrit|alvará|cadastro|licen|trib|certif|dss|cmf', flags = re.I)
tax = np.where(mask9, 'taxas, tarifas e impostos', expenses['favorecido'])
expenses.loc[:,'favorecido'] = tax

outros_mask = ~(expenses['favorecido'].isin(['salários e auxílios alimentação', 'suprimentos', 'medicações e veterinário', 'hospedagem e manutenção de canis', 'ração e agropecuária', 'bazar, divulgação e tecnologias', 'transporte', 'empréstimos e aplicações', 'taxas, tarifas e impostos']))
expenses.loc[outros_mask,'favorecido'] = "outros"
expenses['favorecido'].value_counts()

transporte                          1330
salários e auxílios alimentação     1025
taxas, tarifas e impostos            724
ração e agropecuária                 656
medicações e veterinário             536
outros                               321
hospedagem e manutenção de canis     244
suprimentos                          203
bazar, divulgação e tecnologias       82
Name: favorecido, dtype: int64

In [9]:
# create categories for income

mask = income['depositante'].str.contains('doaç|ração|v.d.|\d|DD|cielo|cielo|pag|pic', regex = True, flags = re.I)
doa = np.where(mask, 'doações e vendas', income['depositante'])
income.loc[:,'depositante'] = doa

mask2 = income['depositante'].str.contains('empr|apli|resg|rend', regex = True, flags = re.I)
emp = np.where(mask2, 'empréstimos e aplicações', income['depositante'])
income.loc[:,'depositante'] = emp

mask3 = income['depositante'].str.contains('Pista', regex = True, flags = re.I)
proj = np.where(mask3, 'projeto Autopista Litoral Sul', income['depositante'])
income.loc[:,'depositante'] = proj

mask4 = income['depositante'].str.contains('justiça|PMF ', regex = True, flags = re.I)
gov = np.where(mask4, 'governo', income['depositante'])
income.loc[:,'depositante'] = gov

mask5 = income['depositante'].str.contains('div|tran|int|seguro|rog|BB|dif', regex = True, flags = re.I)
out = np.where(mask5, 'outros', income['depositante'])
income.loc[:,'depositante'] = out

In [10]:
# final categories for 'expenses'
expenses['favorecido'].value_counts()

transporte                          1330
salários e auxílios alimentação     1025
taxas, tarifas e impostos            724
ração e agropecuária                 656
medicações e veterinário             536
outros                               321
hospedagem e manutenção de canis     244
suprimentos                          203
bazar, divulgação e tecnologias       82
Name: favorecido, dtype: int64

In [11]:
# final categories for 'income'
income['depositante'].value_counts()

doações e vendas                 1710
empréstimos e aplicações          411
projeto Autopista Litoral Sul      31
governo                            14
outros                             10
Name: depositante, dtype: int64

In [12]:
#converting data types
expenses['despesas'] = expenses['despesas'].astype(str).str.strip()
expenses['despesas'] = expenses['despesas'].str.replace(',', '')
expenses['despesas'] = expenses['despesas'].astype(float)
expenses['ano'] = expenses['ano'].astype(int)
income['receita'] = income['receita'].astype(str).str.strip()
income['receita'] = income['receita'].str.replace(',', '')
income['receita'] = income['receita'].astype(float)
income['ano'] = income['ano'].astype(int)


In [14]:
income

,ano,mês,dia,receita,depositante
0,2016,Sep,1.0,565.00,doações e vendas
1,2016,Sep,2.0,350.00,doações e vendas
2,2016,Sep,5.0,360.00,doações e vendas
3,2016,Sep,6.0,140.00,doações e vendas
4,2016,Sep,8.0,440.00,doações e vendas
...,...,...,...,...,...
2171,2022,Nov,28.0,310.00,doações e vendas
2172,2022,Nov,28.0,144.25,empréstimos e aplicações
2173,2022,Nov,29.0,260.00,doações e vendas
2174,2022,Nov,30.0,30.00,doações e vendas


# Exploratory analysis

In [12]:
expenses.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5121 entries, 0 to 5120
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ano         5121 non-null   int32  
 1   mês         5121 non-null   object 
 2   dia         5114 non-null   float64
 3   despesas    5121 non-null   float64
 4   favorecido  5121 non-null   object 
dtypes: float64(2), int32(1), object(2)
memory usage: 180.2+ KB


In [13]:
income.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2176 entries, 0 to 2175
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   ano          2176 non-null   int32  
 1   mês          2176 non-null   object 
 2   dia          2175 non-null   float64
 3   receita      2176 non-null   float64
 4   depositante  2176 non-null   object 
dtypes: float64(2), int32(1), object(2)
memory usage: 76.6+ KB


In [14]:
expenses.head()

,ano,mês,dia,despesas,favorecido
0,2016,Sep,1.0,2000.00,salários e auxílios alimentação
1,2016,Sep,2.0,363.00,ração e agropecuária
2,2016,Sep,2.0,10.00,transporte
3,2016,Sep,2.0,143.57,transporte
4,2016,Sep,2.0,50.00,outros


In [15]:
income.head()

,ano,mês,dia,receita,depositante
0,2016,Sep,1.0,565.0,doações e vendas
1,2016,Sep,2.0,350.0,doações e vendas
2,2016,Sep,5.0,360.0,doações e vendas
3,2016,Sep,6.0,140.0,doações e vendas
4,2016,Sep,8.0,440.0,doações e vendas


# Querying for metrics

## Total income and expenses / year (2020-2022)

In [16]:
# total income (2020-2022);
income20_22 = income[income['ano'] >= 2020]
income20_22[['receita', 'ano']].groupby(by ='ano').sum()

,receita
ano,
2020,164518.09
2021,204584.25
2022,209768.74


In [17]:
# total expenses (2020-2022);
expenses20_22 = expenses[expenses['ano'] >= 2020]
expenses20_22[['despesas', 'ano']].groupby(by ='ano').sum()

,despesas
ano,
2020,166005.92
2021,204564.25
2022,209749.74


## Annual and monthly average income / source  (2020-2022)

In [18]:
# annual average income/source (2020-2022)
def annual_avg(x):
    return round(sum(x)/3, 2)
income20_22.groupby(['depositante']).aggregate({'receita': annual_avg}).sort_values(by = 'receita', ascending = True)

,receita
depositante,
outros,134.18
empréstimos e aplicações,38923.67
doações e vendas,153899.18


In [19]:
# monthly average income/source (2020-2022)
def month_avg(x):
    return round(sum(x)/(3*12), 2)
income20_22.groupby(['depositante']).aggregate({'receita': month_avg}).sort_values(by = 'receita', ascending = True)

,receita
depositante,
outros,11.18
empréstimos e aplicações,3243.64
doações e vendas,12824.93


In [20]:
# annual average income/source (2020-2022)
income20_22.groupby(['ano', 'depositante']).aggregate({'receita':sum})

receita
ano  depositante                        
2020 doações e vendas          139437.56
     empréstimos e aplicações   24731.76
     outros                       348.77
2021 doações e vendas          157806.43
     empréstimos e aplicações   46724.04
     outros                        53.78
2022 doações e vendas          164453.54
     empréstimos e aplicações   45315.20

## Annual and monthly average expenses / categories (2020-2022)

In [21]:
# annual average expense/category (2020-2022)
expenses20_22.groupby(['favorecido']).aggregate({'despesas': annual_avg}).sort_values(by = 'despesas', ascending = True)

,despesas
favorecido,
"bazar, divulgação e tecnologias",1386.58
suprimentos,1929.71
outros,4507.66
hospedagem e manutenção de canis,8271.22
transporte,24367.49
ração e agropecuária,29323.65
medicações e veterinário,36519.27
"taxas, tarifas e impostos",42384.97
salários e auxílios alimentação,44749.42


In [22]:
# monthly average expense/category (2020-2022)
expenses20_22.groupby(['favorecido']).aggregate({'despesas': month_avg}).sort_values(by = 'despesas', ascending = True)

,despesas
favorecido,
"bazar, divulgação e tecnologias",115.55
suprimentos,160.81
outros,375.64
hospedagem e manutenção de canis,689.27
transporte,2030.62
ração e agropecuária,2443.64
medicações e veterinário,3043.27
"taxas, tarifas e impostos",3532.08
salários e auxílios alimentação,3729.12


In [23]:
# annual average expense/category (2020-2022)
expenses20_22.groupby(['ano', 'favorecido']).aggregate({'despesas':sum})

despesas
ano  favorecido                                
2020 bazar, divulgação e tecnologias    2360.00
     hospedagem e manutenção de canis   8571.03
     medicações e veterinário          32984.10
     outros                             5366.59
     ração e agropecuária              30919.37
     salários e auxílios alimentação   33707.01
     suprimentos                        1617.61
     taxas, tarifas e impostos         30577.50
     transporte                        19902.71
2021 bazar, divulgação e tecnologias     695.98
     hospedagem e manutenção de canis   8563.57
     medicações e veterinário          31298.24
     outros                             4171.41
     ração e agropecuária              29681.08
     salários e auxílios alimentação   55542.14
     suprimentos                        1717.74
     taxas, tarifas e impostos         49866.66
     transporte                        23027.43
2022 bazar, divulgação e tecnologias    1103.75
     hospedagem e manutenção de canis   7679.06
     medicações e veterinário          45275.47
     outros                             3984.99
     ração e agropecuária              27370.49
     salários e auxílios alimentação   44999.10
     suprimentos                        2453.78
     taxas, tarifas e impostos         46710.76
     transporte                        30172.34

## Total assisted animals, treatments and costs / animal (2016-2019)

In [24]:
# total assisted animals (2016-2019) 
len(animals)

1069

In [25]:
# yearly average cost/animal 2016-2019
# custos totais necessários para alimentação, castração, vacinação, tratamentos, internações, socorro a acidentes (cobras, ouriços, envenenamentos, atropelamentos, brigas)  
expenses16_19 = expenses[(expenses['ano'] == 2016) | (expenses['ano'] == 2017) | (expenses['ano'] == 2018) | (expenses['ano'] == 2019)]
expenses16_19['despesas'].sum()/4/1026

331.1411111111111

In [26]:
# total castrations (2016-2019) 
len(animals[animals['castrado'] == 'sim'])

428

In [27]:
# total treatments (2016-2019) 
animals_dis = animals[['tratamento de sarna (nº de aplicações) total', 'tratamento pulga (nº de aplicações) total', 'vacinas total',  'vermifugo\ntotal', 'dias de internação (total)']]
animals_dis.columns = ['tratamentos sarna', 'tratamentos pulga', 'vacinas', 'vermífugos', 'dias de internação']
total_care = round(animals_dis.sum())
care_df = pd.DataFrame(total_care)

In [28]:
# total treatments/animal (2016-2019) 
animal_1 = round(total_care/1026)
animal_1
care_df['média/animal'] = animal_1
care_df.columns = ['total', 'média/animal']
care_df['média/animal'] = care_df['média/animal'].astype(int)
care_df

,total,média/animal
tratamentos sarna,16068,16
tratamentos pulga,15980,16
vacinas,1864,2
vermífugos,6019,6
dias de internação,6414,6


# Saving dataframes to csv files for Tableau visualization

In [29]:
# turn all days into '1' and merge date columns
income['dia'] = '1'
income['data'] = pd.to_datetime(income['dia'].fillna(1).astype(int).astype(str) + '-' + income['mês'].astype(str) + '-' +  income['ano'].astype(str))
income.drop(['ano', 'mês', 'dia'], inplace = True, axis = 1)
income

,receita,depositante,data
0,565.00,doações e vendas,2016-09-01
1,350.00,doações e vendas,2016-09-01
2,360.00,doações e vendas,2016-09-01
3,140.00,doações e vendas,2016-09-01
4,440.00,doações e vendas,2016-09-01
...,...,...,...
2171,310.00,doações e vendas,2022-11-01
2172,144.25,empréstimos e aplicações,2022-11-01
2173,260.00,doações e vendas,2022-11-01
2174,30.00,doações e vendas,2022-11-01


In [30]:
# turn all days into '1' and merge date columns
expenses['dia'] = '1'
expenses['data'] = pd.to_datetime(expenses['dia'].fillna(1).astype(int).astype(str) + '-' + expenses['mês'].astype(str) + '-' +  expenses['ano'].astype(str))
expenses.drop(['ano', 'mês', 'dia'], inplace = True, axis = 1)
expenses

,despesas,favorecido,data
0,2000.00,salários e auxílios alimentação,2016-09-01
1,363.00,ração e agropecuária,2016-09-01
2,10.00,transporte,2016-09-01
3,143.57,transporte,2016-09-01
4,50.00,outros,2016-09-01
...,...,...,...
5116,354.25,medicações e veterinário,2022-11-01
5117,260.00,"taxas, tarifas e impostos",2022-11-01
5118,62.90,ração e agropecuária,2022-11-01
5119,28.40,medicações e veterinário,2022-11-01


In [31]:
expenses.to_excel('expenses.xlsx', index = False, sheet_name = 'expenses')

In [32]:
income.to_excel('income.xlsx', index = False, sheet_name = 'income')

In [33]:
care_df.to_excel('care.xlsx', sheet_name = 'care')